In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
N_centr = 5
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="../VTXalignment/input/"
file_names=["my-m_ee_Run14AuAu_87th_new_19865_1065runs.root"] 
file_names=["my-m_ee_Run14AuAu_107th_new_19943_350runs.root"]
#m_ee_Run14AuAu_107th_new_19943_350runs
#m_ee_Run14AuAu_105th_new_19929_255runs m_ee_Run14AuAu_106th_new_19941_1051runs
#m_ee_Run14AuAu_103rd_new_19927_255runs m_ee_Run14AuAu_104th_new_19928_489runs
#m_ee_Run14AuAu_100th_new_19922_447runs m_ee_Run14AuAu_101st_new_19924_207runs
#m_ee_Run14AuAu_98th_new_19918_1064runs m_ee_Run14AuAu_99th_new_19920_1063runs
sim_file_path= "../sim/output/summer25/electron/"
dalitz_file_names=['piminus_50M_new_v8.root']# piminus_50M_v8 piminus_50M_new_v3
dalitz_file_names = ["dalitz_v08.root"]
ccbar_name = ['conv_v08.root']#dalitz_v00
sim_file_names = [dalitz_file_names[0], ccbar_name[0]]
legend_names = ["real","#pi^{-}","c#bar{c}"] 
hist_select_3D_names = ["DCA2_hist_0","DCA2_hist_1","DCA2_hist_2","DCA2_hist_3","DCA2_hist_4","sDCA2_hist_0","sDCA2_hist_1","sDCA2_hist_2","sDCA2_hist_3","sDCA2_hist_4",\
                        "DCA_2D_hist_0","DCA_2D_hist_1","DCA_2D_hist_2","DCA_2D_hist_3","DCA_2D_hist_4","sDCA_2D_hist_0","sDCA_2D_hist_1","sDCA_2D_hist_2","sDCA_2D_hist_3","sDCA_2D_hist_4",\
                            "centr_hist"]

In [3]:
N_hists = len(hist_select_3D_names)
loc = True
Save_to_html = False

In [4]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


for iFile in range(len(sim_file_names)):
    print(sim_file_path+sim_file_names[iFile])
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)

infile.Close()

../VTXalignment/input/my-m_ee_Run14AuAu_107th_new_19943_350runs.root
../sim/output/summer25/electron/dalitz_v08.root
../sim/output/summer25/electron/conv_v08.root


In [5]:
N_ev = []
for icentr in range(N_centr):
    N_ev.append(hists_read[0][-1].ProjectionY(f"N_ev_{icentr}",icentr*20+1,(icentr+1)*20+1).Integral())
    print(f"Number of events in {icentr} centrality bin = {N_ev[-1]/1e9:.2f} B")
RAA = [0.4,0.5,0.7,0.9,0.9]
RAA = [x/0.5 for x in RAA]
Ncoll = [771/2,282.4,82.6,12.1,3]

Number of events in 0 centrality bin = 0.64 B
Number of events in 1 centrality bin = 0.66 B
Number of events in 2 centrality bin = 0.65 B
Number of events in 3 centrality bin = 0.62 B
Number of events in 4 centrality bin = 0.33 B


In [18]:
conv_type = [16,25]
pt = [0.35,0.35]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
centr_length = [0.2/0.93,0.2/0.93,0.2/0.93,0.2/0.93,0.13/0.93]
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+10
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real_prodfjproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    piminus_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[0][type_loc].GetName()+f"dalitz_dfprojproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(piminus_proj,Lcolor=4, Msize=0)
    conv_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"ccbar_dfprojproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2, Msize=0)
    real_proj.Scale(1./N_ev[icentr])
    #pi_scale  = 1./(1+piminus_proj.Integral())/3
    #piminus_proj.Scale(pi_scale)
    piminus_proj.Scale(10/(centr_length[icentr]*25.e6)*RAA[icentr]) # 0.2 is the bin width in pT
    conv_proj.Scale(10/(centr_length[icentr]*250e6)*RAA[icentr]/0.01174)
    sum_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"copycat{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    for ibin in range(1, sum_proj.GetNbinsX()+1):
        sum_proj.SetBinContent(ibin,  piminus_proj.GetBinContent(ibin) + conv_proj.GetBinContent(ibin))
        sum_proj.SetBinError(ibin, ( piminus_proj.GetBinError(ibin)**2 + conv_proj.GetBinError(ibin)**2)**0.5)
    Format_Hist_total(sum_proj,Lcolor=6, Msize=0)
    print(f"ratio conv/dalitz = {conv_proj.Integral()/(piminus_proj.Integral()+1e-12):.2f},     real/sim = {real_proj.Integral()/(sum_proj.Integral()+1e-12):.2f}")
    #piminus_proj.Scale(1./(1+sum_proj.Integral()))
    #conv_proj.Scale(1./(1+sum_proj.Integral()))
    #sum_proj.Scale(1./(1+sum_proj.Integral()))
    #real_proj.Scale(1./N_ev[icentr])
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(1e-10*Ncoll[icentr],2e-5*Ncoll[icentr])
    real_proj.Draw("HIST")
    piminus_proj.Draw("sameH")
    conv_proj.Draw("sameH")
    sum_proj.Draw("sameH")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(piminus_proj,legend_names[1],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    # Calculate and print standard deviation
    #print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    #print(f"Standard deviation for dalitz_proj in centrality bin {icentr}: {piminus_proj.GetStdDev()}")
    #print(f"beam width = {(real_proj.GetStdDev()**2-piminus_proj.GetStdDev()**2)**0.5}" )

c0.Draw()
c0.SaveAs(f"output/dca/sim.png")

ratio conv/dalitz = 2.37,     real/sim = 0.67
ratio conv/dalitz = 1.16,     real/sim = 0.86
ratio conv/dalitz = 0.52,     real/sim = 0.71
ratio conv/dalitz = 0.41,     real/sim = 0.87
ratio conv/dalitz = 0.32,     real/sim = 0.73


In [7]:
conv_type = [16,25]
pt = [.45,.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+10
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real1_prodfjproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    piminus_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[0][type_loc].GetName()+f"dal1itz_dfprojproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(piminus_proj,Lcolor=4, Msize=0)
    conv_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"ccb1ar_dfprojproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2, Msize=0)
    real_proj.Scale(1./(1+real_proj.Integral()))
    #pi_scale  = 1./(1+piminus_proj.Integral())/3
    #piminus_proj.Scale(pi_scale)
    conv_scale = 25/250/0.01174
    conv_proj.Scale(conv_scale)
    print(f"ratio conv/dalitz = {conv_proj.Integral()/piminus_proj.Integral():.2f}")
    sum_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"copy1cat{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    for ibin in range(1, sum_proj.GetNbinsX()+1):
        sum_proj.SetBinContent(ibin,  piminus_proj.GetBinContent(ibin) + conv_proj.GetBinContent(ibin))
        sum_proj.SetBinError(ibin, ( piminus_proj.GetBinError(ibin)**2 + conv_proj.GetBinError(ibin)**2)**0.5)
    Format_Hist_total(sum_proj,Lcolor=6, Msize=0)
    piminus_proj.Scale(1./(1+sum_proj.Integral()))
    conv_proj.Scale(1./(1+sum_proj.Integral()))
    sum_proj.Scale(1./(1+sum_proj.Integral()))
    #real_proj.Scale(1./N_ev[icentr])
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(0.000011,20)
    real_proj.Draw("HIST")
    piminus_proj.Draw("sameH")
    conv_proj.Draw("sameH")
    sum_proj.Draw("sameH")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(piminus_proj,legend_names[1],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    # Calculate and print standard deviation
    #print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    #print(f"Standard deviation for dalitz_proj in centrality bin {icentr}: {piminus_proj.GetStdDev()}")
    #print(f"beam width = {(real_proj.GetStdDev()**2-piminus_proj.GetStdDev()**2)**0.5}" )

c0.Draw()
c0.SaveAs(f"output/dca/sim.png")

ratio conv/dalitz = 2.98
ratio conv/dalitz = 1.42
ratio conv/dalitz = 0.68
ratio conv/dalitz = 0.37
ratio conv/dalitz = 0.34


In [8]:
conv_type = [1,25]
pt = [1.45,4.55]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+5
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"rsdeal_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    piminus_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dasdlitz_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(piminus_proj,Lcolor=4, Msize=0)
    ccbar_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"ccbar_dfprojproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=2, Msize=0)
    real_proj.Scale(1./(1+real_proj.Integral()))
    piminus_proj.Scale(1./(1+piminus_proj.Integral()))
    ccbar_proj.Scale(1./(1+ccbar_proj.Integral()))  
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(0.000011,20)
    real_proj.Fit("gaus", "", "", -120, 120)
    real_proj.Draw("HIST")
    piminus_proj.Draw("sameH")
    #ccbar_proj.Draw("sameH")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(piminus_proj,legend_names[1],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    # Calculate and print standard deviation
    print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    print(f"Standard deviation for simul in centrality bin {icentr}: {piminus_proj.GetStdDev()}")
    print(f"beam width = {(real_proj.GetStdDev()**2-piminus_proj.GetStdDev()**2)**0.5}" )

c0.Draw()
c0.SaveAs(f"output/dca/sim.png")

Standard deviation for real_proj in centrality bin 0: 104.20734536778951
Standard deviation for simul in centrality bin 0: 81.54547291370584
beam width = 64.8807111234289
Standard deviation for real_proj in centrality bin 1: 104.04549257436176
Standard deviation for simul in centrality bin 1: 101.31680992051525
beam width = 23.672104945943232
Standard deviation for real_proj in centrality bin 2: 110.0267169575627
Standard deviation for simul in centrality bin 2: 93.90580389164498
beam width = 57.33740873045737
Standard deviation for real_proj in centrality bin 3: 107.26490924099744
Standard deviation for simul in centrality bin 3: 86.53135729160476
beam width = 63.38836612306754
Standard deviation for real_proj in centrality bin 4: 125.60182378539955
Standard deviation for simul in centrality bin 4: 104.39009319641845
beam width = 69.84645002189896
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      75.6078
NDf                       =

In [9]:
conv_type = [1,25]
pt = [1.45,4.55]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"fprojy{0}")
projz = hists_read[0][0].ProjectionZ(f"ffprojz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+0
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"rsdeadl_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    piminus_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dasddlitz_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(piminus_proj,Lcolor=4, Msize=0)
    real_proj.Scale(1./(1+real_proj.Integral()))
    piminus_proj.Scale(1./(1+piminus_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(0.000011,20)
    real_proj.Fit("gaus", "", "", -120, 120)
    real_proj.Draw("HIST")
    piminus_proj.Draw("sameH")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(piminus_proj,legend_names[1],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    # Calculate and print standard deviation
    print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    print(f"Standard deviation for simul in centrality bin {icentr}: {piminus_proj.GetStdDev()}")
    print(f"beam width = {(real_proj.GetStdDev()**2-piminus_proj.GetStdDev()**2)**0.5}" )

c0.Draw()
c0.SaveAs(f"output/dca/sim.png")

Standard deviation for real_proj in centrality bin 0: 104.0883071724828
Standard deviation for simul in centrality bin 0: 87.1680849818634
beam width = 56.8867352783386
Standard deviation for real_proj in centrality bin 1: 101.40739544381992
Standard deviation for simul in centrality bin 1: 75.61722457744104
beam width = 67.5684482425349
Standard deviation for real_proj in centrality bin 2: 107.82311953248556
Standard deviation for simul in centrality bin 2: 85.73214488985585
beam width = 65.38978848644055
Standard deviation for real_proj in centrality bin 3: 104.87585343410299
Standard deviation for simul in centrality bin 3: 85.28669149735549
beam width = 61.03380118398627
Standard deviation for real_proj in centrality bin 4: 122.0718608616538
Standard deviation for simul in centrality bin 4: 96.45685291775722
beam width = 74.81720884548633
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      75.6476
NDf                       =      

In [10]:
infile = root.TFile.Open(sim_file_path+sim_file_names[0], "read")
hist_pt_mother= infile.Get("hist_pt_mother")
hist_pt_mother.SetDirectory(root.nullptr)
hist_pt_mother_weight = infile.Get("hist_pt_mother_weight")
hist_pt_mother_weight.SetDirectory(root.nullptr)
infile.Close()

In [11]:
c00 = root.TCanvas(f"c00",f"c00",1550,900)
hagedorn = root.TF1("hagedorn", " 2 * 3.1415 * x * [0] * pow( exp(-[1]*x-[2]*x*x)+x/[3] , -[4])", 100, 0.4, 10)
hagedorn.SetParameters(377, 0.356, 0.068, 0.7,8.25)
hagedorn.SetLineWidth(3)
hagedorn.SetLineColor(2)
hagedorn.SetNpx(1000)
hist_pt_mother_porj = hist_pt_mother.ProjectionX("hist_pt_mother_proj")
Format_Hist_total(hist_pt_mother_porj, title_x="p_{T}, GeV/c", title_y="dN/dp_{T}, GeV/c^{-1}", offset_y=1.1, Mcolor=1, Msize =1, Lcolor=1, Lwidth=5)
hist_pt_mother_porj.Draw("P")
for ibin in range(1, hist_pt_mother_porj.GetNbinsX()+1):
    hist_pt_mother_porj.SetBinError(ibin, (hist_pt_mother_porj.GetBinContent(ibin)**0.5+0.2*hist_pt_mother_porj.GetBinContent(ibin))/hist_pt_mother_porj.GetBinWidth(ibin)/25e6*9.6*1.5)
    hist_pt_mother_porj.SetBinContent(ibin, hist_pt_mother_porj.GetBinContent(ibin)/hist_pt_mother_porj.GetBinWidth(ibin)/25e6*9.6*1.5)
#hist_pt_mother_porj.Fit(hagedorn, "R", "", 0.4, 10)
hagedorn.Draw("same")
root.gPad.SetLogy()
c00.Draw()

In [12]:
c00 = root.TCanvas(f"c00",f"c00",1550,900)
hagedorn = root.TF1("hagedorn", " 2 * 3.1415 * x * [0] * pow( exp(-[1]*x-[2]*x*x)+x/[3] , -[4])", 100, 0.4, 10)
hagedorn.SetParameters(377, 0.356, 0.068, 0.7,8.25)
hagedorn.SetLineWidth(3)
hagedorn.SetLineColor(2)
hist_pt_mother_porj = hist_pt_mother_weight.ProjectionX("hist_pt_mother_weight_proj")
Format_Hist_total(hist_pt_mother_porj, title_x="p_{T}, GeV/c", title_y="dN/dp_{T}, GeV/c^{-1}", offset_y=1.1, Mcolor=1, Msize =1, Lcolor=1, Lwidth=5)
hist_pt_mother_porj.Draw("P")
for ibin in range(1, hist_pt_mother_porj.GetNbinsX()+1):
    hist_pt_mother_porj.SetBinError(ibin, (hist_pt_mother_porj.GetBinContent(ibin)**0.5+0.2*hist_pt_mother_porj.GetBinContent(ibin))/hist_pt_mother_porj.GetBinWidth(ibin)/25e6*9.6*10)
    hist_pt_mother_porj.SetBinContent(ibin, hist_pt_mother_porj.GetBinContent(ibin)/hist_pt_mother_porj.GetBinWidth(ibin)/25e6*9.6*10)
#hist_pt_mother_porj.Fit(hagedorn, "R", "", 0.4, 10)
hagedorn.Draw("same")
root.gPad.SetLogy()
c00.Draw()